In [5]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from metrics import dice_loss, dice_coef, iou
from train import create_dir

""" Global parameters """
H = 512
W = 512

In [8]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir("test_images\\mask")

""" Loading model """
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
    model = tf.keras.models.load_model("files\\model.h5")

""" Load dataset """
data_x = glob("test_images\\image\\*")

for path in tqdm(data_x, total=len(data_x)):
  """ Extract the name """
  name = path.split("\\")[-1].split(".")[0]

  """ Reading the image """
  image = cv2.imread(path, cv2.IMREAD_COLOR)
  h, w, _ = image.shape
  x = cv2.resize(image, (W, H))
  x = x/255.0
  x = x.astype(np.float32)
  x = np.expand_dims(x, axis=0)

  """ Prediction """
  y = model.predict(x)[0]
  y = cv2.resize(y, (w, h))
  y = np.expand_dims(y, axis=-1)

  """ Save the image """
  masked_image = image * y
  line = np.ones((h, 10, 3)) * 128
  cat_images = np.concatenate([image, line, masked_image], axis=1)
  cv2.imwrite(f"test_images\\mask\\{name}.png", cat_images)

100%|██████████| 5/5 [00:09<00:00,  1.82s/it]
